In [0]:
%sql
SELECT * FROM ds_training_1.ds_control.table_config; 

query freq

In [0]:
%sql
create or replace table ds_training_1.ds_gold.imp_queries_query_freq (statement_text string, query_frequency string)

In [0]:
%sql
INSERT INTO ds_training_1.ds_gold.imp_queries_query_freq
SELECT 
    statement_text, 
    COUNT(statement_text) AS query_frequency
FROM system.query.history
WHERE statement_text LIKE '%observe_ai.ecomm_sales%'
GROUP BY statement_text;

In [0]:
%sql
select * from ds_training_1.ds_gold.imp_queries_query_freq


imp query data


In [0]:
%sql
create or replace table ds_training_1.ds_gold.imp_queries_data (statement_id string, executed_by string, statement_text string, execution_duration_ms bigint, execution_status string, error_message string, read_files bigint, end_time timestamp)

In [0]:
%sql
INSERT INTO ds_training_1.ds_gold.imp_queries_data
SELECT 
    h.statement_id, 
    h.executed_by, 
    h.statement_text, 
    h.execution_duration_ms, 
    h.execution_status,
    h.error_message, 
    h.read_files,
    h.end_time

FROM 
    system.query.history h
JOIN 
    (SELECT 
        '%' || CONCAT(catalog_name, '.', schema_name, '.', table_name) || '%' AS pattern
     FROM 
        ds_training_1.ds_control.table_config) tc
ON 
    h.statement_text LIKE tc.pattern;


In [0]:
%sql
select * from ds_training_1.ds_gold.imp_queries_data ;

In [0]:
%sql
select statement_id, executed_by, statement_text, execution_duration_ms, read_files, execution_status, error_message, end_time from system.query.history where statement_text like '%observe_ai.ecomm_sales%' ;

In [0]:
%sql
-- %sql
-- INSERT INTO ds_training_1.ds_gold.imp_queries_data
-- SELECT 
--     statement_id, 
--     executed_by, 
--     statement_text, 
--     execution_duration_ms, 
--     read_files 
-- FROM system.query.history 
-- WHERE statement_text LIKE '%observe_ai.ecomm_sales%';


In [0]:
%sql
select * from ds_training_1.ds_gold.imp_queries_data
order by end_time desc

query frequency

In [0]:
%sql
create table ds_training_1.ds_gold.imp_queries_query_freq(statement_text string, query_frequency int)

In [0]:
%sql
INSERT INTO  ds_training_1.ds_gold.imp_queries_query_freq(statement_text, query_frequency)
SELECT 
    statement_text, 
    count(statement_text) as query_frequency
FROM 
    ds_training_1.ds_gold.imp_queries_data 
WHERE 
    statement_text LIKE '%observe_ai.ecomm_sales%'
GROUP BY 
    statement_text 
ORDER BY 
    query_frequency DESC;


In [0]:
%sql
select * from ds_training_1.ds_gold.imp_queries_query_freq

In [0]:
%sql
-- %sql
-- select statement_text, count(statement_text) as query_frequency  from ds_training_1.ds_gold.imp_queries_data where statement_text like '%observe_ai.ecomm_sales%' group by statement_text order by query_frequency desc;

user frequency

In [0]:
%sql
select executed_by, count(executed_by) as user_frequency from ds_training_1.ds_gold.imp_queries_data where statement_text like '%observe_ai.ecomm_sales%'  group by executed_by order by count(executed_by) desc;

important queries ranked with normalized composite score

In [0]:
%sql
create or replace table ds_training_1.ds_gold.imp_queries_rank(statement_text string, executed_by string, execution_status string,error_message string, composite_score double, rank int)

In [0]:
%sql
select * from ds_training_1.ds_gold.imp_queries_rank

failed important queries

In [0]:
%sql
create table ds_training_1.ds_gold.important_queries_failed(statement_text string, executed_by string, execution_status string, error_message string, composite_score double, rank int)

In [0]:
%sql
select * from ds_training_1.ds_gold.important_queries_failed

classified imp _queries_

In [0]:
%sql
create table ds_training_1.ds_gold.important_queries_classification(statement_text string, executed_by string, execution_status string, error_message string, composite_score string, rank int, query_importance string)

In [0]:
%sql
select * from ds_training_1.ds_gold.important_queries_classification

bad queries


In [0]:
%sql
create or replace table ds_training_1.ds_gold.bad_queries_rank(statement_text string, executed_by string, execution_status string, error_message string, normalized_composite_score double, rank int)

In [0]:
%sql
select * from ds_training_1.ds_gold.bad_queries_rank

In [0]:
%sql
create or replace table ds_training_1.ds_gold.bad_queries_data(statement_id string, executed_by string, statement_text string, total_duration_ms bigint, execution_duration_ms bigint, compilation_duration_ms bigint, read_rows bigint, produced_rows bigint, read_bytes bigint,end_time timestamp, normalized_total_duration_ms double, normalized_execution_duration_ms double, normalized_compilation_duration_ms double, normalized_read_rows double, normalized_produced_rows double, normalized_read_bytes double)

In [0]:
%sql
select * from ds_training_1.ds_gold.bad_queries_data